In [ ]:
from pathlib import Path
from models.transfersdataset import TransfersDataset
from models.catboost import CatBoost

In [ ]:
import pandas as pd
import warnings

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
td.encode_last_positions()
td.encode()
td.drop()

In [ ]:
att_features = ['sattact90','goals90','npgoals90','xg90','hdrgoals90','shots90','ontrgtpct','goalcnvrsnpct','assists90','crosses90',
                'acccrossespct','crossesleft90','acccrossesleftpct','crossesright90','acccrossesrightpct','crosses6yard90','dribbles90',
                'succssdribblespct','offduels90','offduelspct','touchesbox90','prgruns90','acclr90','rcvdpasses90','rcvdlongpasses90',
                'foulsa90','fwdpasses90','accfwdpassespct','xa90','shotassist90','secassist90','thirdassist90','smartpasses90',
                'accsmartpassespct','kp90','final3rdpasses90','accfinal3rdpassespct','penareapasses90','accpenareapassespct','thrpasses90',
                'accthrpassespct','directfk90','directfkontrgtpct','corners90','pens','pencnvrspct']
att_features2 = ['goals90','npgoals90','xg90','hdrgoals90','shots90','xa90','shotassist90','touchesbox90','prgruns90','acclr90','rcvdpasses90','smartpasses90', 'dribbles90',
                'succssdribblespct']

In [ ]:
def pipeline(pos, features=None, full_feature_set=False):
    td = TransfersDataset(Path.cwd()/'../prepped/final-2.csv')
    td.filter_postion(pos)
    td.encode()
    td.drop()
    m = CatBoost(data = td.data, features = features, full_feature_set=full_feature_set)
    display(m.data[m.features], ', '.join(m.features))
    m.train_test_split()
    m.scale()
    m.tune_hp()
    m.train()
    m.inverse_scale()
    m.score(thresh=10)
    # display(m.feature_importance())
    m.plot_predictions()
    display(m.top_n_predictions(20))
    display(m.top_n_predictions(20, worst=True))

In [ ]:
pipeline('ATT')

In [ ]:
pipeline('ATT')

In [ ]:
pipeline('ATT', att_features2, full_feature_set=False)

In [ ]:
scores = pd.read_csv('catboost_att.csv').iloc[:,1:].sort_values('MSE')
scores.Features = scores.Features.str.split(', ')
feature_sets = scores.Features.head(10).to_dict()

In [ ]:
for _, features in feature_sets.items():
    pipeline(pos='ATT', features=features, full_feature_set=True)